In [ ]:
# Universal Setup for Backend Environment
import sys
import os
import subprocess
from pathlib import Path

def setup_environment():
    """Setup the environment by installing necessary dependencies and setting paths."""
    # Get the backend directory. If we are in 'backend', it is cwd.
    backend_dir = Path.cwd()
    if backend_dir.name != 'backend':
        # Search for backend
        if (backend_dir / 'backend').exists():
             backend_dir = backend_dir / 'backend'
        elif (backend_dir.parent / 'backend').exists():
             backend_dir = backend_dir.parent / 'backend'
    
    # Add src to path if it exists (for 'from agent import ...' style)
    src_dir = backend_dir / 'src'
    if src_dir.exists():
        if str(src_dir) not in sys.path:
            sys.path.append(str(src_dir))
            print(f"✅ Added {src_dir} to sys.path")
    
    if str(backend_dir) not in sys.path:
        sys.path.append(str(backend_dir))
        
    # Verify backend/agent can be imported
    try:
        import agent
        print("✅ Agent module found and imported.")
    except ImportError:
        print("❌ Agent module not found. Installing dependencies...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", str(backend_dir)])
        print("✅ Backend installed in editable mode.")

setup_environment()

# 🏛️ Agent Architecture Demo

This notebook visualizes the internal structure of the LangGraph agent and demonstrates its execution flow.

## Features
- **Graph Visualization**: See the nodes and edges of the agent
- **Step-by-Step Execution**: Trace how the agent processes a query
- **State Inspection**: View the agent's internal state at each step

---

In [ ]:
# === COLAB SETUP (Auto-detects environment) ===
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("🔧 Running in Google Colab - Setting up environment...")
    
    # Clone repository
    !git clone https://github.com/MasumRab/gemini-fullstack-langgraph-quickstart --depth 1
    %cd gemini-fullstack-langgraph-quickstart/backend
    
    # Clean conflicting packages
    !pip uninstall -y google-ai-generativelanguage google-generativeai tensorflow grpcio-status 2>/dev/null || true
    
    # Install backend dependencies
    !pip install -q .
    
    # Set API Key from Colab secrets
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
        print("✅ API Key loaded from Colab Secrets")
    except:
        print("⚠️ API Key not found in secrets. Please set it manually:")
        os.environ["GEMINI_API_KEY"] = input("Enter your Gemini API Key: ")
else:
    print("💻 Running locally - Assuming dependencies are installed")
    import os
    import sys
    # Add backend/src to path for local development
    sys.path.insert(0, os.path.abspath("../backend/src"))

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the Gemini model to use. Only Gemini 2.5 models are currently accessible via the API.

MODEL_STRATEGY = "Gemini 2.5 Flash (Recommended)" # @param ["Gemini 2.5 Flash (Recommended)", "Gemini 2.5 Flash-Lite (Fastest)", "Gemini 2.5 Pro (Best Quality)"]

import os

# Map selection to model ID
# Note: Gemini 1.5 and 2.0 models are deprecated/not accessible via this API
if MODEL_STRATEGY == "Gemini 2.5 Flash (Recommended)":
    SELECTED_MODEL = "gemini-2.5-flash"
elif MODEL_STRATEGY == "Gemini 2.5 Flash-Lite (Fastest)":
    SELECTED_MODEL = "gemini-2.5-flash-lite"
elif MODEL_STRATEGY == "Gemini 2.5 Pro (Best Quality)":
    SELECTED_MODEL = "gemini-2.5-pro"
else:
    # Default fallback
    SELECTED_MODEL = "gemini-2.5-flash"

print(f"Selected Model: {SELECTED_MODEL}")
print(f"Strategy: {MODEL_STRATEGY}")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

In [ ]:
# --- MODEL VERIFICATION (Optional) ---
# @title Verify Model Configuration
# @markdown Run this cell to verify that your API key is configured correctly and the selected model is available.

import os

# Check if API key is set
if "GEMINI_API_KEY" not in os.environ:
    print("⚠️  GEMINI_API_KEY not found in environment variables!")
    print("   Please set it before proceeding:")
    print("   export GEMINI_API_KEY='your-api-key-here'")
else:
    try:
        from google import genai
        
        # Initialize the client
        client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])
        
        # Test the selected model
        print(f"🧪 Testing model: {SELECTED_MODEL}")
        response = client.models.generate_content(
            model=SELECTED_MODEL,
            contents="Explain how AI works in a few words"
        )
        
        print(f"✅ Model verification successful!")
        print(f"   Model: {SELECTED_MODEL}")
        print(f"   Response: {response.text[:100]}...")
        
    except ImportError:
        print("⚠️  google-genai package not installed!")
        print("   Installing now...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-genai"])
        print("✅ Installed! Please re-run this cell.")
        
    except Exception as e:
        print(f"❌ Model verification failed: {e}")
        print(f"   This could mean:")
        print(f"   - Invalid API key")
        print(f"   - Model '{SELECTED_MODEL}' not available in your region")
        print(f"   - Quota/billing issues (for experimental models)")
        print(f"   - Network connectivity issues")

## 1. Load the Agent Graph

We import the compiled graph object from the backend code.

In [ ]:
# Import the graph
try:
    from agent.graph import graph
    print("✅ Graph imported successfully")
except ImportError as e:
    print(f"❌ Failed to import graph: {e}")
    print("Ensure you are in the correct directory (backend/) and dependencies are installed.")

## 2. Visualize the Architecture

We use Mermaid.js to render the graph structure directly in the notebook.

In [ ]:
from IPython.display import Image, display

print("📊 Generating Graph Visualization...")

try:
    # Draw the graph as a PNG
    png_data = graph.get_graph().draw_mermaid_png()
    display(Image(png_data))
    print("✅ Visualization complete")
except Exception as e:
    print(f"⚠️ Graphical visualization failed: {e}")
    print("\nFalling back to ASCII representation:")
    print("="*40)
    graph.get_graph().print_ascii()
    print("="*40)

## 3. Run a Demo Query

Execute a simple query to see the agent in action.

In [ ]:
from langchain_core.messages import HumanMessage
import asyncio

async def run_demo(question: str):
    """Run the agent and stream output."""
    print(f"🤖 Processing Query: \"{question}\"")
    print("-" * 50)

    config = {"configurable": {"thread_id": "demo_notebook"}}
    
    async for event in graph.astream_events(
        {"messages": [HumanMessage(content=question)]},
        config,
        version="v2"
    ):
        kind = event["event"]
        
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="", flush=True)
                
        elif kind == "on_tool_start":
            print(f"\n\n🛠️ Executing Tool: {event['name']}")
            
        elif kind == "on_tool_end":
            print(f"✅ Tool Completed: {event['name']}\n")

    print("\n" + "-" * 50)
    print("🏁 Execution Complete")

# Run the demo
QUERY = "What is the capital of France?"
await run_demo(QUERY)

---
## 📚 Learn More

- **Visualization Guide**: `docs/guides/VISUALIZATION_EXAMPLES.md`
- **LangGraph Docs**: [https://langchain-ai.github.io/langgraph/](https://langchain-ai.github.io/langgraph/)